In [1]:
%matplotlib widget
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
from signal_alignment import phase_align, chisqr_align
from dateutil.parser import *

In [2]:
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', 10000)

In [3]:
referenceFile = pd.read_csv('./UVVis/BareSolarLamp2.csv')
referenceFile['intensity'] = referenceFile['intensity'].rolling(50, win_type='nuttall').mean().shift(-25).fillna(1)

spectrumBaseline = referenceFile[referenceFile.wavelength > 200]
spectrumBaseline = spectrumBaseline[referenceFile.wavelength < 238]
spectrumBaseline = spectrumBaseline['intensity'].mean()
referenceFile['intensity'] = referenceFile['intensity'] - spectrumBaseline


referenceFileMaxValue = referenceFile[referenceFile.wavelength > 823.15]
referenceFileMaxValue = referenceFileMaxValue[referenceFile.wavelength < 823.359]
referenceFileMaxValue = referenceFileMaxValue['intensity'].mean()

referenceFile['intensityNorm'] = referenceFile['intensity'] / referenceFileMaxValue

referenceFile = referenceFile.set_index('wavelength')

In [4]:
files = [
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure2.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure3.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure4.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure5.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure6.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure7.csv',
    './UVVis/UVVisCell_Sample04_50nmSi_SolarGen/UVVis/InitProcedure8.csv'
]

dataframes = []

for fileName in files:
    file = pd.read_csv(fileName)
    file['time'] = pd.to_datetime(file['time'], unit='s')
    file = file.set_index([pd.DatetimeIndex(file['time'])], drop=True)
    file = file.drop(columns='time')
    dataframes.append(file)
    

file = pd.concat(dataframes)

In [5]:
potentiostatData = pd.read_csv('./UVVis/UVVisCell_Sample04_50nmSi_SolarGen/Potentiostate/MergedFile.csv')
# potentiostatData['actualTime'] = potentiostatData['actualTime'].map(lambda x: parse(x).timestamp())
potentiostatData['actualTime'] = pd.to_datetime(potentiostatData['actualTime'])
# file['time'] = pd.to_datetime(file['time'], unit='s')
potentiostatData = potentiostatData.set_index('actualTime')
potentiostatData['T'] = potentiostatData.index - potentiostatData.index.values[0]

In [6]:
mergedDataframe = pd.merge_asof(file, potentiostatData, left_index = True, right_index = True, tolerance=pd.Timedelta("1 minutes"))
mergedDataframe = mergedDataframe.dropna(how='any')

In [7]:
figure2 = plt.figure()

figure2ax1 = figure2.add_subplot(1,2,1)
# figure2ax1.set_ylim(0, 1.5)
figure2ax2 = figure2.add_subplot(1,2,2)
# figure2ax2.set_ylim(-0.5, 6)



X = mergedDataframe[str(mergedDataframe.index.values[0])]['wavelength'].values
Y = [date for date in mergedDataframe.groupby('time').groups]

XX, YY = np.meshgrid(X, Y)

Z_absSmoothed = []
Z_intensityNorm = []
Z_Im = []
Z_V = []

for yGrid in YY:
    try:
        spectrum = mergedDataframe[str(yGrid[0])]
        spectrum = spectrum.set_index('wavelength')
        
        spectrum['intensity'] = spectrum['intensity'].rolling(50).mean()

        spectrumBaseline = spectrum[(spectrum.index > 200) & (spectrum.index < 300)]
        spectrumBaseline = spectrumBaseline['intensity'].mean()
        spectrum['intensity'] = spectrum['intensity'] - spectrumBaseline


        spectrumMaxValue = spectrum[(spectrum.index > 823.15) & (spectrum.index < 823.359)]
        spectrumMaxValue = spectrumMaxValue['intensity'].mean()
        spectrum['intensityNorm'] = spectrum['intensity'] / spectrumMaxValue


        spectrum['abs'] = -np.log(spectrum['intensity'] / referenceFile['intensityNorm'])
        spectrum['absSmoothed'] = spectrum['abs'].rolling(50).mean()

        figure2ax1.plot(spectrum.index, spectrum['intensityNorm'], label=spectrum['n'].min())
        figure2ax2.plot(spectrum.index, spectrum['absSmoothed'])

        Z_absSmoothed.append(spectrum['absSmoothed'].values)
        Z_intensityNorm.append(spectrum['intensityNorm'].values)
        Z_Im.append(spectrum['Im'].values)
        Z_V.append(spectrum['Vf'].values)
        
    
    except KeyError as err:
        continue
         


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
figure3 = plt.figure()

figure3ax1 = figure3.add_subplot(1,4,1)
# ax5.set_xlim(265, 807)

figure3ax2 = figure3.add_subplot(1,4,2)
# ax6.set_xlim(265, 807)

figure3ax3 = figure3.add_subplot(1,4,3)
# ax6.set_xlim(265, 807)

figure3ax4 = figure3.add_subplot(1,4,4)
# ax6.set_xlim(265, 807)

cont = figure3ax1.pcolormesh(XX, YY, Z_absSmoothed, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=-12, vmax=-2)

cont = figure3ax2.pcolormesh(XX, YY, Z_intensityNorm, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=0, vmax=3.18)

cont = figure3ax3.pcolormesh(XX, YY, Z_V, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto', vmin=0.01, vmax=3.2)

cont = figure3ax4.pcolormesh(XX, YY, Z_Im, cmap=cm.plasma,
                       linewidth=0, antialiased=True, shading='auto')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
potentiostatData.plot(x='T', y='Im')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='T'>

In [10]:
mergedDataframe

,Unnamed: 0_x,wavelength,intensity,acqTime,n,Unnamed: 0_y,T,Vf,Im
time,,,,,,,,,
2021-01-18 14:30:00.459413504,0,178.997010,2826.292840,5581,62,36.0,0 days 00:00:36,3.195240,-4.228770e-08
2021-01-18 14:30:00.459413504,1,179.215281,2826.292840,5581,62,36.0,0 days 00:00:36,3.195240,-4.228770e-08
2021-01-18 14:30:00.459413504,2,179.433542,2826.292840,5581,62,36.0,0 days 00:00:36,3.195240,-4.228770e-08
2021-01-18 14:30:00.459413504,3,179.651791,2874.498720,5581,62,36.0,0 days 00:00:36,3.195240,-4.228770e-08
2021-01-18 14:30:00.459413504,4,179.870030,2854.859287,5581,62,36.0,0 days 00:00:36,3.195240,-4.228770e-08
...,...,...,...,...,...,...,...,...,...
2021-01-18 19:01:29.471059968,3643,884.693799,5463.332970,23400,189,886.0,0 days 04:32:05,0.356823,-2.735890e-06
2021-01-18 19:01:29.471059968,3644,884.858069,5363.350406,23400,189,886.0,0 days 04:32:05,0.356823,-2.735890e-06
2021-01-18 19:01:29.471059968,3645,885.022319,5234.801395,23400,189,886.0,0 days 04:32:05,0.356823,-2.735890e-06


In [11]:
mergedDataframeReindexed = mergedDataframe.reset_index().set_index(['T', 'wavelength'])

In [12]:
mergedDataframeReindexed

time  Unnamed: 0_x  \
T               wavelength                                               
0 days 00:00:36 178.997010 2021-01-18 14:30:00.459413504             0   
                179.215281 2021-01-18 14:30:00.459413504             1   
                179.433542 2021-01-18 14:30:00.459413504             2   
                179.651791 2021-01-18 14:30:00.459413504             3   
                179.870030 2021-01-18 14:30:00.459413504             4   
...                                                  ...           ...   
0 days 04:32:05 884.693799 2021-01-18 19:01:29.471059968          3643   
                884.858069 2021-01-18 19:01:29.471059968          3644   
                885.022319 2021-01-18 19:01:29.471059968          3645   
                885.186551 2021-01-18 19:01:29.471059968          3646   
                885.350764 2021-01-18 19:01:29.471059968          3647   

                              intensity  acqTime    n  Unnamed: 0_y        Vf  \
T               wavelength                                                      
0 days 00:00:36 178.997010  2826.292840     5581   62          36.0  3.195240   
                179.215281  2826.292840     5581   62          36.0  3.195240   
                179.433542  2826.292840     5581   62          36.0  3.195240   
                179.651791  2874.498720     5581   62          36.0  3.195240   
                179.870030  2854.859287     5581   62          36.0  3.195240   
...                                 ...      ...  ...           ...       ...   
0 days 04:32:05 884.693799  5463.332970    23400  189         886.0  0.356823   
                884.858069  5363.350406    23400  189         886.0  0.356823   
                885.022319  5234.801395    23400  189         886.0  0.356823   
                885.186551  5234.801395    23400  189         886.0  0.356823   
                885.350764  5234.801395    23400  189         886.0  0.356823   

                                      Im  
T               wavelength                
0 days 00:00:36 178.997010 -4.228770e-08  
                179.215281 -4.228770e-08  
                179.433542 -4.228770e-08  
                179.651791 -4.228770e-08  
                179.870030 -4.228770e-08  
...                                  ...  
0 days 04:32:05 884.693799 -2.735890e-06  
                884.858069 -2.735890e-06  
                885.022319 -2.735890e-06  
                885.186551 -2.735890e-06  
                885.350764 -2.735890e-06  

[4093056 rows x 8 columns]

In [13]:
mergedDataframeReindexed.xs('0 days 00:00:36')

,time,Unnamed: 0_x,intensity,acqTime,n,Unnamed: 0_y,Vf,Im
wavelength,,,,,,,,
178.997010,2021-01-18 14:30:00.459413504,0,2826.292840,5581,62,36.0,3.19524,-4.228770e-08
179.215281,2021-01-18 14:30:00.459413504,1,2826.292840,5581,62,36.0,3.19524,-4.228770e-08
179.433542,2021-01-18 14:30:00.459413504,2,2826.292840,5581,62,36.0,3.19524,-4.228770e-08
179.651791,2021-01-18 14:30:00.459413504,3,2874.498720,5581,62,36.0,3.19524,-4.228770e-08
179.870030,2021-01-18 14:30:00.459413504,4,2854.859287,5581,62,36.0,3.19524,-4.228770e-08
180.088258,2021-01-18 14:30:00.459413504,5,2772.730752,5581,62,36.0,3.19524,-4.228770e-08
180.306475,2021-01-18 14:30:00.459413504,6,2844.146870,5581,62,36.0,3.19524,-4.228770e-08
180.524681,2021-01-18 14:30:00.459413504,7,2831.649049,5581,62,36.0,3.19524,-4.228770e-08
180.742877,2021-01-18 14:30:00.459413504,8,2837.005258,5581,62,36.0,3.19524,-4.228770e-08


In [14]:
mergedDataframeReindexed.xs('0 days 04:32:05').plot(y='intensity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='wavelength'>